In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE134470"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE134470"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE134470.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE134470.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE134470.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis reveals close resemblance between Glioblastoma (GBM) patient tumors and corresponding patient-derived orthotopic xenografts (PDOXs)"
!Series_summary	"Glioblastoma (GBM) patient-derived orthotopic xenografts (PDOXs) were derived from organotypic spheroids obtained from patient tumor samples.  To detect whether gene expression profiles of GBM patient tumors are retained in PDOXs, we performed genome-wide transcript analysis by human-specific microarrays . In parallel, we analyzed GBM cell cultures and corresponding intracranial xenografts from stem-like (NCH421k, NCH644) and adherent GBM cell lines (U87, U251). PDOXs show a better transcriptomic resemblance with patient tumors than other preclinical models. The major difference is largely explained by the depletion of human-derived non-malignant cells."
!Series_overall_design	"58 samples from human GBM patient tumor samples (n=6), GBM PDOXs (6 PDOX models, n=1-3), GBM cell l

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable

# Review the dataset information
# 1. Gene Expression Data Availability
# From the background information, we see this is GeneChip® Human Gene 1.0ST affymetrix array data
# which contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Glioblastoma) - From the characteristics, we can infer from row 0 (tissue/cell type)
trait_row = 0

# For age - No information about age in the characteristics dictionary
age_row = None

# For gender - No information about gender in the characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert tissue/cell type to binary trait values (0: control, 1: GBM)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Normal brain is the control (0), all GBM samples are cases (1)
    if 'normal brain' in value.lower():
        return 0
    elif 'gbm' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous or binary format."""
    # Not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary format (0: female, 1: male)."""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical data - in this case, we need to use the provided path
    try:
        # Try to load existing clinical data file if it exists
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
        else:
            # No clinical data CSV available, create an empty DataFrame with appropriate columns
            # This is a common situation: the function needs input but there's no actual data
            print("No clinical data file found. Creating minimal dataset.")
            # Create a minimal clinical data DataFrame that has the required format
            clinical_data = pd.DataFrame(columns=[trait_row]) 

        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        # Create an empty clinical features file to maintain workflow
        pd.DataFrame(columns=[trait]).to_csv(out_clinical_data_file, index=False)
        print(f"Empty clinical data saved to: {out_clinical_data_file}")


No clinical data file found. Creating minimal dataset.
Error in clinical feature extraction: Length mismatch: Expected axis has 0 elements, new values have 1 elements
Empty clinical data saved to: ../../output/preprocess/Glioblastoma/clinical_data/GSE134470.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM3949211"	"GSM3949212"	"GSM3949213"	"GSM3949214"	"GSM3949215"	"GSM3949216"	"GSM3949217"	"GSM3949218"	"GSM3949219"	"GSM3949220"	"GSM3949221"	"GSM3949222"	"GSM3949223"	"GSM3949224"	"GSM3949225"	"GSM3949226"	"GSM3949227"	"GSM3949228"	"GSM3949229"	"GSM3949230"	"GSM3949231"	"GSM3949232"	"GSM3949233"	"GSM3949234"	"GSM3949235"	"GSM3949236"	"GSM3949237"	"GSM3949238"	"GSM3949239"	"GSM3949240"	"GSM3949241"	"GSM3949242"	"GSM3949243"	"GSM3949244"	"GSM3949245"	"GSM3949246"	"GSM3949247"	"GSM3949248"	"GSM3949249"	"GSM3949250"	"GSM3949251"	"GSM3949252"	"GSM3949253"	"GSM3949254"	"GSM3949255"	"GSM3949256"	"GSM3949257"	"GSM3949258"	"GSM3949259"	"GSM3949260"	"GSM3949261"	"GSM3949262"	"GSM3949263"	"GSM3949264"	"GSM3949265"	"GSM3949266"	"GSM3949267"	"GSM3949268"
First data line: 7892501	5.654938132	5.721028692	5.67452351	5.641837111	5.743193368	5.566262298	5.56729464	5.636138876	5.635626425	5.950472587	5.881945937	5.954507095	6.063136572	5.888634637	5.67

Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Inspecting the gene identifiers in the expression data
# Based on the numbers in the ID column like "7892501", "7892502", etc., 
# these appear to be probe IDs from a microarray platform, not human gene symbols
# These numerical IDs need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE134470
Line 6: !Series_title = Gene expression analysis reveals close resemblance between Glioblastoma (GBM) patient tumors and corresponding patient-derived orthotopic xenografts (PDOXs)
Line 7: !Series_geo_accession = GSE134470
Line 8: !Series_status = Public on Jul 01 2020
Line 9: !Series_submission_date = Jul 18 2019
Line 10: !Series_last_update_date = Dec 22 2020
Line 11: !Series_pubmed_id = 33009951
Line 12: !Series_pubmed_id = 33311477
Line 13: !Series_summary = Glioblastoma (GBM) patient-derived orthotopic xenografts (PDOXs) were derived from organotypic spheroids obtained from patient tumor samples.  To detect whether gene expression profiles of GBM patient tumors are retained in PDOXs, we perfo


Gene annotation preview:
{'ID': [7896736, 7896738, 7896740, 7896742, 7896744], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7, 31, 24, 6, 36], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe which column in gene_annotation relates to the gene expression data identifiers
# From previous outputs, we see gene expressions have identifiers like "7892501", "7892502", etc.
# Looking at the gene annotation dataframe, the "ID" column contains similar numeric identifiers

# Define the columns to extract from the gene annotation data
probe_id_col = "ID"  # The identifier in gene expression data
gene_symbol_col = "gene_assignment"  # The gene symbols/annotations

# 2. Extract mapping between probes and genes
mapping_df = gene_annotation[[probe_id_col, gene_symbol_col]]
mapping_df = mapping_df.rename(columns={gene_symbol_col: 'Gene'})
mapping_df = mapping_df.astype({'ID': 'str'})  # Ensure ID is string type to match expression data index

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the result shows gene symbols instead of probe IDs
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")


Gene expression data after mapping:
Shape: (117474, 58)
First 5 gene symbols: ['A-', 'A-3-', 'A-52', 'A-E', 'A-I']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using data from previous step
# We already have gene_data from the previous step
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Generate clinical data
# Based on previous output, we know we only have the trait information
# All samples are glioblastoma as indicated by the tissue info in sample characteristics

# First, let's create a proper clinical dataframe based on what we know
# We need a dataframe with sample IDs as columns and trait as the only row
clinical_df = pd.DataFrame(index=[trait], 
                          data={col: 1 for col in normalized_gene_data.columns})
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis. No demographic features available."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (24229, 58)
Sample gene symbols after normalization: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE134470.csv
Clinical data shape: (1, 58)
Clinical data preview:
              GSM3949211  GSM3949212  GSM3949213  GSM3949214  GSM3949215  \
Glioblastoma           1           1           1           1           1   

              GSM3949216  GSM3949217  GSM3949218  GSM3949219  GSM3949220  ...  \
Glioblastoma           1           1           1           1           1  ...   

              GSM3949259  GSM3949260  GSM3949261  GSM3949262  GSM3949263  \
Glioblastoma           1           1           1           1           1   

              GSM3949264  GSM3949265  GSM3949266  GSM3949267  GSM3949268  
Glioblastoma           1           1           1           1           1  

[1 rows x 58 columns]
Clinical data saved to ../../output/preprocess/Glioblastoma/clinical_data/GSE134470.csv
Linked data shape: (58, 24230)
Linked data preview (first 5 rows, first 5 columns):
            Glioblastoma      A1BG      A1C

Data shape after handling missing values: (58, 24230)
Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
